# Evaluation of TF-IDF Model on Queries

This notebook evaluates a TF-IDF model on a set of queries and qrels by calculating the Mean Average Precision (MAP), Mean Precision at Recall (MPR), and Precision.

In [ ]:
# Import necessary librariesimport pandas as pdimport joblibfrom sklearn.feature_extraction.text import TfidfVectorizerfrom sklearn.metrics.pairwise import cosine_similarityfrom collections import defaultdictimport numpy as np

In [ ]:
# Load the preprocessed documents and queriesDATA_PATH = '/content/drive/MyDrive/downloads'DOCS_FILE = f'{DATA_PATH}/documents.tsv'QUERIES_FILE = f'{DATA_PATH}/queries.tsv'QRELS_FILE = f'{DATA_PATH}/qrels.tsv'
docs_df = pd.read_csv(DOCS_FILE, sep='	')queries_df = pd.read_csv(QUERIES_FILE, sep='	')qrels_df = pd.read_csv(QRELS_FILE, sep='	')

In [ ]:
# Define evaluation metricsdef calculate_average_precision(retrieved_docs, relevant_docs):    if not relevant_docs:        return 0.0    hits = 0    sum_precisions = 0.0    for i, doc_id in enumerate(retrieved_docs):        if doc_id in relevant_docs:            hits += 1            precision_at_k = hits / (i + 1)            sum_precisions += precision_at_k    return sum_precisions / len(relevant_docs)
def calculate_precision_at_k(retrieved_docs, relevant_docs, k):    relevant_retrieved = len(set(retrieved_docs[:k]) & relevant_docs)    return relevant_retrieved / k
def calculate_mean_precision_at_recall(retrieved_docs, relevant_docs):    recall_levels = np.linspace(0.0, 1.0, num=11)    precisions = []    recalls = []    relevant_retrieved = 0    for i, doc_id in enumerate(retrieved_docs):        if doc_id in relevant_docs:            relevant_retrieved += 1        precision = relevant_retrieved / (i + 1)        recall = relevant_retrieved / len(relevant_docs)        precisions.append(precision)        recalls.append(recall)    precision_at_recall = {}    for recall_level in recall_levels:        max_precision = 0.0        for i, recall in enumerate(recalls):            if recall >= recall_level:                max_precision = max(max_precision, precisions[i])        precision_at_recall[recall_level] = max_precision    return precision_at_recall

In [ ]:
# Evaluate Modelprint('Running Evaluation...')
# Example setup for TF-IDF vectorizer (should be replaced with actual model loading)tfidf_vectorizer = joblib.load('/content/drive/MyDrive/tfidf-optimized/tfidf_vectorizer.joblib')tfidf_matrix = joblib.load('/content/drive/MyDrive/tfidf-optimized/tfidf_matrix.joblib')doc_ids = joblib.load('/content/drive/MyDrive/tfidf-optimized/doc_ids.joblib')
# Perform evaluationaverage_precisions = []precision_at_10 = []recall_at_10 = []for index, query_row in queries_df.iterrows():    query_id = query_row['query_id']    query_text = query_row['text']    relevant_docs = set(qrels_df[qrels_df['query_id'] == query_id]['doc_id'].values)    if len(relevant_docs) == 0:        continue    
    # Process query    query_vector = tfidf_vectorizer.transform([query_text])    scores = cosine_similarity(query_vector, tfidf_matrix).flatten()    ranked_indices = np.argsort(-scores)    retrieved_docs = [doc_ids[i] for i in ranked_indices[:1000]]
    # Calculate metrics    ap = calculate_average_precision(retrieved_docs, relevant_docs)    average_precisions.append(ap)    precision_at_10.append(calculate_precision_at_k(retrieved_docs, relevant_docs, 10))    
# Display resultsmean_ap = np.mean(average_precisions)print(f'Mean Average Precision (MAP): {mean_ap}')print(f'Mean Precision at 10: {np.mean(precision_at_10)}')